In [1]:
import slopestabilitytools
import slopestabilityML
import settings

import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

### Read and preprocess data

In [2]:
test_results = slopestabilitytools.datamanagement.import_tests()
is_success = slopestabilitytools.folder_structure.create_folder_structure()
random_seed = 999
test_training, test_prediction = slopestabilityML.split_dataset(test_results.keys(), random_seed)
num_feat = []
if settings.settings['norm'] is True:
    num_feat.append('RESN')
else:
    num_feat.append('RES')

if settings.settings['sen'] is True:
    num_feat.append('SEN')

if settings.settings['depth'] is True:
    num_feat.append('Y')

num_trans = StandardScaler()

if settings.settings['use_labels'] is True:
    cat_feat = ['LABELS']
    cat_lab = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
    cat_trans = OneHotEncoder(categories=[cat_lab])
    preprocessor = ColumnTransformer(transformers=[('num', num_trans, num_feat)])
    # ('cat', cat_trans, cat_feat)])

else:
    preprocessor = ColumnTransformer(transformers=[('num', num_trans, num_feat)])



test_results_combined = pd.DataFrame()
for name in test_training:
    test_results_combined = test_results_combined.append(test_results[name])
test_results_combined = test_results_combined.reset_index()
test_results_combined = test_results_combined.drop(['index'], axis='columns')
x_train, y_train = slopestabilityML.preprocess_data(test_results_combined)
x_position = test_results_combined['X']

AttributeError: module 'settings' has no attribute 'settings'

### SVM

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
svc_parameters = {}
svc_classifier = svm.SVC()
svc_pipeline = make_pipeline(preprocessor, svc_classifier)
clf = GridSearchCV(svc_pipeline, svc_parameters)
clf.fit(x_train, y_train)
sorted(clf.cv_results_.keys())